In [2]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [4]:
df = pd.read_csv('diabetes.csv') 

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Отбор первых 590 строк:

In [6]:
task_data = df.head(590)

Вывод числа строк в получившейся выборке и относящихся к классу 1 (пациент болен диабетом)

In [7]:
len(task_data[task_data['Outcome'] == 1])

204

Разделеные данных на тренировочные и тестовые. Первые 70% строк — тренировочные, остальные — тестовые.

In [8]:
train = task_data.head(int(len(task_data)*0.7))
test = task_data.tail(int(len(task_data)*0.3))

Выделяем предикторы (8 первых столбцов) и отклик (Outcome):

In [10]:
features = list(train.columns[:8])
x = train[features]
y = train['Outcome']

Подключем классификатор DecisionTreeClassifier:

In [11]:
from sklearn.tree import DecisionTreeClassifier

Задаем параметры дерева принятия решений и обучаем модель:

In [12]:
tree = DecisionTreeClassifier(criterion='entropy', #критерий разделения
                              min_samples_leaf=20, #минимальное число объектов в листе
                              max_leaf_nodes=30, #максимальное число листьев
                              random_state=2020)
clf=tree.fit(x, y)

Подключаем библиотеку для визуализации дерева. Сохраняем в файл и выводим на экран.

In [17]:
from sklearn.tree import export_graphviz
import graphviz
columns = list(x.columns)
export_graphviz(clf, out_file='tree.dot', 
                feature_names=columns,
                class_names=['0', '1'],
                rounded = True, proportion = False, 
                precision = 2, filled = True, label='all')

with open('tree.dot') as f:
    dot_graph = f.read()

graphviz.Source(dot_graph)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

Выводим глубину дерева:

In [16]:
clf.tree_.max_depth

7

Выполняем предсказание для объектов из тестовой выборки:

In [13]:
features = list(test.columns[:8])
x = test[features]
y_true = test['Outcome']
y_pred = clf.predict(x)

Доля правильных ответов классификатора:

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.7909604519774012

Среднее значение метрик $F_1$ (Macro-F1):

In [15]:
from sklearn.metrics import f1_score
f1_score(y_true, y_pred, average='macro')

0.7301940427635645

Выполним предсказание для конкретного объекта (с индексом 708) исходных данных:

In [16]:
df.loc[708, features]

Pregnancies                   9.000
Glucose                     164.000
BloodPressure                78.000
SkinThickness                 0.000
Insulin                       0.000
BMI                          32.800
DiabetesPedigreeFunction      0.148
Age                          45.000
Name: 708, dtype: float64

Назначенный класс:

In [18]:
clf.predict([df.loc[708, features].tolist()])[0]

1